# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords

# import libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)

import sys
import re
from sqlalchemy import create_engine
import pickle

# import relevant functions/modules from the sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
database_filepath = 'DisasterResponse.db'
engine = create_engine('sqlite:///'+ database_filepath)
df = pd.read_sql_table('DisasterResponse', engine)

# Remove child alone as it has all zeros only
df = df.drop(['child_alone'],axis=1)

# Given value 2 in the related field are neglible so it could be error. Replacing 2 with 1 to consider it a valid response
# Alternatively, we could have assumed it to be 0 also. In the absence of information I have gone with majority class
df['related']=df['related'].map(lambda x: 1 if x == 2 else x)

X = df.message
y = df[df.columns[4:]]
category_names = y.columns

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text,url_place_holder_string="urlplaceholder"):
    """
    tokenize
    Tokenize the text function
    
    Input:
    text -> Text message which needs to be tokenized
    
    Returns:
    clean_tokens -> List of tokens extracted from the provided text
    """ 
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    # Remove stop words
    stop_words = stopwords.words("english")
    tokens = [tok for tok in tokens if tok not in stop_words]
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC())))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = pipeline.predict(X_test) # predict
class_report = classification_report(y_test, y_pred, target_names=category_names) # print classification report
print(class_report)
print('Accuracy: {}'.format(np.mean(y_test.values == y_pred))) # print accuracy score

                        precision    recall  f1-score   support

               related       0.86      0.91      0.89      3997
               request       0.75      0.59      0.66       875
                 offer       0.00      0.00      0.00        26
           aid_related       0.69      0.71      0.70      2098
          medical_help       0.58      0.31      0.41       429
      medical_products       0.62      0.26      0.36       292
     search_and_rescue       0.65      0.17      0.27       129
              security       0.20      0.01      0.02        81
              military       0.60      0.35      0.44       168
                 water       0.70      0.55      0.62       303
                  food       0.78      0.68      0.73       548
               shelter       0.71      0.51      0.59       436
              clothing       0.77      0.42      0.54        79
                 money       0.49      0.20      0.28       125
        missing_people       0.67      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
#model parameters for GridSearchCV
parameters = {'vect__ngram_range': ((1, 1), (1, 2)),
                  'vect__max_df': (0.75, 1.0)
                  }

cv = GridSearchCV(estimator=pipeline,
            param_grid=parameters,
            verbose=3,
            cv=3)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] vect__max_df=0.75, vect__ngram_range=(1, 1) .....................
[CV]  vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.27606923186954657, total=  17.6s
[CV] vect__max_df=0.75, vect__ngram_range=(1, 1) .....................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   27.8s remaining:    0.0s


[CV]  vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.28279216135030755, total=  17.5s
[CV] vect__max_df=0.75, vect__ngram_range=(1, 1) .....................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   55.5s remaining:    0.0s


[CV]  vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.27968526466380544, total=  17.5s
[CV] vect__max_df=0.75, vect__ngram_range=(1, 2) .....................
[CV]  vect__max_df=0.75, vect__ngram_range=(1, 2), score=0.28221999713917895, total=  21.7s
[CV] vect__max_df=0.75, vect__ngram_range=(1, 2) .....................
[CV]  vect__max_df=0.75, vect__ngram_range=(1, 2), score=0.28365040766700045, total=  21.7s
[CV] vect__max_df=0.75, vect__ngram_range=(1, 2) .....................
[CV]  vect__max_df=0.75, vect__ngram_range=(1, 2), score=0.28240343347639485, total=  21.7s
[CV] vect__max_df=1.0, vect__ngram_range=(1, 1) ......................
[CV]  vect__max_df=1.0, vect__ngram_range=(1, 1), score=0.2759261908167644, total=  17.7s
[CV] vect__max_df=1.0, vect__ngram_range=(1, 1) ......................
[CV]  vect__max_df=1.0, vect__ngram_range=(1, 1), score=0.28365040766700045, total=  17.6s
[CV] vect__max_df=1.0, vect__ngram_range=(1, 1) ......................
[CV]  vect__max_df=1.0, v

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  6.0min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'vect__max_df': (0.75, 1.0)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [15]:
# Get the prediction values from the grid search cross validator
y_test = cv.predict(X_test)
y_train = cv.predict(X_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
class_report = classification_report(y_test, y_pred, target_names=category_names) # print classification report
print(class_report)

#print('Accuracy: {}'.format(np.mean(y_test.values == y_pred))) # print accuracy score

                        precision    recall  f1-score   support

               related       0.98      0.94      0.96      4397
               request       0.89      0.80      0.84       764
                 offer       0.00      0.00      0.00         0
           aid_related       0.92      0.84      0.88      2361
          medical_help       0.84      0.75      0.79       260
      medical_products       0.72      0.74      0.73       117
     search_and_rescue       0.82      0.80      0.81        35
              security       0.60      1.00      0.75         3
              military       0.86      0.82      0.84       101
                 water       0.90      0.74      0.82       288
                  food       0.95      0.85      0.90       535
               shelter       0.92      0.80      0.85       359
              clothing       0.91      0.89      0.90        44
                 money       0.82      0.91      0.87        46
        missing_people       0.80      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [22]:
m = pickle.dumps('classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.